# Agenda
1. Chatbot Overview with Langchain
2. Spotify Songs Recommender with faiss vector database
  - dataset: https://www.kaggle.com/datasets/joebeachcapital/30000-spotify-songs?select=spotify_songs.csv

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.0 Chatbot Overview with Langchain

- source: <strong>langchain/chatbots</strong> with modification

## Use case

Chatbots are one of the central LLM use-cases. The core features of chatbots are that they can have long-running conversations and have access to information that users want to know about.

Aside from basic prompting and LLMs, memory and retrieval are the core components of a chatbot. Memory allows a chatbot to remember past interactions, and retrieval provides a chatbot with up-to-date, domain-specific information.

## Overview

The chat model interface is based around messages rather than raw text. Several components are important to consider for chat:

* `chat model`: See [here](/docs/integrations/chat) for a list of chat model integrations and [here](/docs/modules/model_io/chat) for documentation on the chat model interface in LangChain. You can use `LLMs` (see [here](/docs/modules/model_io/llms)) for chatbots as well, but chat models have a more conversational tone and natively support a message interface.
* `prompt template`: Prompt templates make it easy to assemble prompts that combine default messages, user input, chat history, and (optionally) additional retrieved context.
* `memory`: [See here](/docs/modules/memory/) for in-depth documentation on memory types
* `retriever` (optional): [See here](/docs/modules/data_connection/retrievers) for in-depth documentation on retrieval systems. These are useful if you want to build a chatbot with domain-specific knowledge.

## Quickstart

Here's a quick preview of how we can create chatbot interfaces. First let's install some dependencies and set the required credentials:

In [2]:
!pip install langchain openai tiktoken chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [3]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

## Regular Conversation

In [20]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [5]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [6]:
# Prompt
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ]
)

In [7]:
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [8]:
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

In [9]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi how are you"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi how are you

> Finished chain.


{'question': 'hi how are you',
 'chat_history': [HumanMessage(content='hi how are you'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?")],
 'text': "Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"}

In [10]:
conversation(
    {"question": "my name is niken, i want to ask you something"}
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi how are you
AI: Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?
Human: my name is niken, i want to ask you something

> Finished chain.


{'question': 'my name is niken, i want to ask you something',
 'chat_history': [HumanMessage(content='hi how are you'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"),
  HumanMessage(content='my name is niken, i want to ask you something'),
  AIMessage(content="Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you.")],
 'text': "Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you."}

In [11]:
conversation({"question": "do you know natural language processing (NLP) ?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi how are you
AI: Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?
Human: my name is niken, i want to ask you something
AI: Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you.
Human: do you know natural language processing (NLP) ?

> Finished chain.


{'question': 'do you know natural language processing (NLP) ?',
 'chat_history': [HumanMessage(content='hi how are you'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"),
  HumanMessage(content='my name is niken, i want to ask you something'),
  AIMessage(content="Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you."),
  HumanMessage(content='do you know natural language processing (NLP) ?'),
  AIMessage(content='Yes, I am familiar with natural language processing (NLP). NLP is a field of artificial intelligence that focuses on the interaction between computers and human language. It involves analyzing, understanding, and generating natural language to enable computers to comprehend and respond to human language in a meaningful way. NLP is used in various applications like chatbots, language translation, sentiment analysis, and

In [12]:
conversation({"question": "what is the different between natural language understanding and natural language generation ?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi how are you
AI: Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?
Human: my name is niken, i want to ask you something
AI: Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you.
Human: do you know natural language processing (NLP) ?
AI: Yes, I am familiar with natural language processing (NLP). NLP is a field of artificial intelligence that focuses on the interaction between computers and human language. It involves analyzing, understanding, and generating natural language to enable computers to comprehend and respond to human language in a meaningful way. NLP is used in various applications like chatbots, language translation, sentiment analysis, and more. Is there anything specific you would like to know about NLP?
Hu

{'question': 'what is the different between natural language understanding and natural language generation ?',
 'chat_history': [HumanMessage(content='hi how are you'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"),
  HumanMessage(content='my name is niken, i want to ask you something'),
  AIMessage(content="Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you."),
  HumanMessage(content='do you know natural language processing (NLP) ?'),
  AIMessage(content='Yes, I am familiar with natural language processing (NLP). NLP is a field of artificial intelligence that focuses on the interaction between computers and human language. It involves analyzing, understanding, and generating natural language to enable computers to comprehend and respond to human language in a meaningful way. NLP is used in various applications like chatbots

In [13]:
conversation({"question": "who is my name ?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi how are you
AI: Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?
Human: my name is niken, i want to ask you something
AI: Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you.
Human: do you know natural language processing (NLP) ?
AI: Yes, I am familiar with natural language processing (NLP). NLP is a field of artificial intelligence that focuses on the interaction between computers and human language. It involves analyzing, understanding, and generating natural language to enable computers to comprehend and respond to human language in a meaningful way. NLP is used in various applications like chatbots, language translation, sentiment analysis, and more. Is there anything specific you would like to know about NLP?
Hu

{'question': 'who is my name ?',
 'chat_history': [HumanMessage(content='hi how are you'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"),
  HumanMessage(content='my name is niken, i want to ask you something'),
  AIMessage(content="Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you."),
  HumanMessage(content='do you know natural language processing (NLP) ?'),
  AIMessage(content='Yes, I am familiar with natural language processing (NLP). NLP is a field of artificial intelligence that focuses on the interaction between computers and human language. It involves analyzing, understanding, and generating natural language to enable computers to comprehend and respond to human language in a meaningful way. NLP is used in various applications like chatbots, language translation, sentiment analysis, and more. Is there anything specif

In [14]:
conversation({"question": "whos win in the battle between gojo satoru vs ryomen sukuna ?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi how are you
AI: Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?
Human: my name is niken, i want to ask you something
AI: Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you.
Human: do you know natural language processing (NLP) ?
AI: Yes, I am familiar with natural language processing (NLP). NLP is a field of artificial intelligence that focuses on the interaction between computers and human language. It involves analyzing, understanding, and generating natural language to enable computers to comprehend and respond to human language in a meaningful way. NLP is used in various applications like chatbots, language translation, sentiment analysis, and more. Is there anything specific you would like to know about NLP?
Hu

{'question': 'whos win in the battle between gojo satoru vs ryomen sukuna ?',
 'chat_history': [HumanMessage(content='hi how are you'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"),
  HumanMessage(content='my name is niken, i want to ask you something'),
  AIMessage(content="Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you."),
  HumanMessage(content='do you know natural language processing (NLP) ?'),
  AIMessage(content='Yes, I am familiar with natural language processing (NLP). NLP is a field of artificial intelligence that focuses on the interaction between computers and human language. It involves analyzing, understanding, and generating natural language to enable computers to comprehend and respond to human language in a meaningful way. NLP is used in various applications like chatbots, language translation, sentiment

## retriever conversation
- add new context about gojo vs sukana

In [15]:
loader = WebBaseLoader("https://beebom.com/jujutsu-kaisen-gojo-vs-sukuna/")
data = loader.load()

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [24]:
embd = embedding=OpenAIEmbeddings()


In [22]:
retriever = vectorstore.as_retriever()
qa_retriever = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [23]:
qa_retriever("whos win in the battle between gojo satoru vs ryomen sukuna ?")

{'question': 'whos win in the battle between gojo satoru vs ryomen sukuna ?',
 'chat_history': [HumanMessage(content='hi how are you'),
  AIMessage(content="Hello! I'm just a chatbot, so I don't have feelings, but I'm here to help you. How can I assist you today?"),
  HumanMessage(content='my name is niken, i want to ask you something'),
  AIMessage(content="Hello Niken! I'm here to answer any questions you may have. Please feel free to ask me anything, and I'll do my best to assist you."),
  HumanMessage(content='do you know natural language processing (NLP) ?'),
  AIMessage(content='Yes, I am familiar with natural language processing (NLP). NLP is a field of artificial intelligence that focuses on the interaction between computers and human language. It involves analyzing, understanding, and generating natural language to enable computers to comprehend and respond to human language in a meaningful way. NLP is used in various applications like chatbots, language translation, sentiment

# 2.0 Similarity Search using Faiss
- https://github.com/facebookresearch/faiss
- https://python.langchain.com/docs/integrations/vectorstores/faiss

In this project, we will utilize the Faiss vector database, a powerful library for similarity search and recommendation, to create our song recommendation system. Faiss allows us to efficiently search through vast amounts of data to find courses that closely match the ones you’ve enjoyed or are interested in.

In [ ]:
!pip install faiss-gpu

In [3]:
import numpy as np
import pandas as pd
import random

import faiss

from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

In [13]:
data = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/NLP_AI_Week_Ruangguru/spotify_songs.csv').dropna().reset_index(drop=True)

In [14]:
data.head(2)

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.00000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.00421,0.3570,0.693,99.972,162600


In [51]:
data_clean = data[['track_name', 'track_artist']].drop_duplicates().reset_index(drop=True)

In [52]:
data_clean.shape

(26229, 2)

In [53]:
#create corpus
data_clean['corpus'] = data_clean['track_artist'] + " " + data_clean['track_name']

song_corpus = data_clean['corpus']

In [54]:
#vectorization
vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,3), min_df=5)
X = vectorizer.fit_transform(song_corpus)

In [55]:
X.shape

(26229, 6183)

In [56]:
#convert sparse matrix to numpy array
X_array = np.float32(X.toarray())

# create vector database index
index = faiss.IndexFlatL2(X_array.shape[1])

# add vectors to the index
index.add(X_array)

In [65]:
# testing search
search_text = ["coldplay"]
search_text_vector = vectorizer.transform(search_text)
search_text_vector_array = np.float32(search_text_vector.toarray())

distances, indices = index.search(search_text_vector_array, 5)

for song_index in indices[0]:
    print(f"Song Title: {data_clean['track_name'][song_index]} from {data_clean['track_artist'][song_index]}")

Song Title: O from Coldplay
Song Title: Ink from Coldplay
Song Title: Talk from Coldplay
Song Title: Fix You from Coldplay
Song Title: Magic from Coldplay


In [66]:
#cerate function
def recommend_course(title):
    search_text = [title]
    search_text_vector = vectorizer.transform(search_text)
    search_text_vector_array = np.float32(search_text_vector.toarray())
    distances, indices = index.search(search_text_vector_array, 5)

    n_1 = f"Song Title: {data_clean['track_name'][indices[0][0]]} from {data_clean['track_artist'][indices[0][0]]}"
    n_2 = f"Song Title: {data_clean['track_name'][indices[0][1]]} from {data_clean['track_artist'][indices[0][1]]}"
    n_3 = f"Song Title: {data_clean['track_name'][indices[0][2]]} from {data_clean['track_artist'][indices[0][2]]}"

    return n_1, n_2, n_3

In [67]:
recommend_course("chainsmokers")

('Song Title: Young from The Chainsmokers',
 'Song Title: Paris from The Chainsmokers',
 'Song Title: Roses from The Chainsmokers')

In [68]:
recommend_course("coldplay")

('Song Title: O from Coldplay',
 'Song Title: Ink from Coldplay',
 'Song Title: Talk from Coldplay')

In [69]:
recommend_course("Denny Caknan")

('Song Title: İz from FG',
 'Song Title: Kenny G from Matuê',
 "Song Title: sunny from j'san")

In [70]:
recommend_course("Happy Asmara")

('Song Title: Happy from ØMG',
 'Song Title: Happy from Ashanti',
 'Song Title: İz from FG')